# 𝕊𝕖𝕞𝕚-𝕊𝕦𝕡𝕖𝕣𝕧𝕚𝕤𝕖𝕕 𝕝𝕖𝕒𝕣𝕟𝕚𝕟𝕘 𝕦𝕤𝕚𝕟𝕘 𝕄𝕖𝕒𝕟 𝕋𝕖𝕒𝕔𝕙𝕖𝕣

Implementation of pixel-wise Mean Teacher (MT)
    
This method is proposed in the paper: 
    'Mean Teachers are Better Role Models:
        Weight-Averaged Consistency Targets Improve Semi-Supervised Deep Learning Results'
This implementation only supports Gaussian noise as input perturbation, and the two-heads
outputs trick is not available.

Source:
https://github.com/ZHKKKe/PixelSSL/blob/master/pixelssl/ssl_algorithm/ssl_mt.py


Todo:
* [] get everything on cuda, cpu -> to configs.device
* [] metrics for segmentation + unittests
* [] track the combined loss - what's the weight, ...


Things that we have to set to the device (gpu):
* the model
* the image that gets fed into the model
* the ground truth that gets fed into the criterion
* the ground truth that gets fed into the metrics - due to activation functions


recall high: most of the roi has been segmented
in combination with low precision: general oversegmentation

Using Learning Rate Scheduler:

https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/5

```
steps = 10
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

for epoch in range(5):
    for idx in range(steps):
        scheduler.step()
        print(scheduler.get_lr())

    print('Reset scheduler')
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)
```            


* TODO
* BINARY_MODE MULTICLASS_MODE - the loss wants a different encoding ...
* ground_truth = torch.nn.functional.one_hot(ground_truth, num_classes)  # N,H*W -> N,H*W, C
* ground_truth = ground_truth.permute(0, 2, 1)  # N, C, H*W
* RuntimeError: one_hot is only applicable to index tensor.

           
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:38: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:38: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
  
  OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:355: error: (-215:Assertion failed) n >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::fitEllipseNoDirect'
  
  C:\Users\Christina\Documents\datasceyence\helper\compute\loss\shape.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(loss)
C:\Users\Christina\anaconda3\envs\chrisy\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')

# Experiments Overview

| Experiment name | model | with cleaning | with FDA | with MUNIT | ramp-up | epoch | f-score | prec | rec | 
|---              |---    |---            |---       |---         |---      |---|---|---|---|
| baseline        | b0 | no  | no  | no  | 300 | 144 | 0.884 | 0.935 | 0.852 |
| FDA             | b0 | yes | no  | no  | 300 | 140 | 0.9 | 0.914 | 0.899 |
| clean             | b0 | yes | yes | no  | 50  | 0 | 1 | 1 | 1 |
| MUNIT           | b0 | yes | no  | yes | 50  | 0 | 1 | 1 | 1 |
| super-net r50   | b7 | yes | yes | no  | 50  | 0 | 1 | 1 | 1 |
| super-net r200  | b7 | yes | yes | no  | 200 | 0 | 1 | 1 | 1 |

'fscore': 0.884, 'jac': 0.798, 'prec': 0.935, 'rec': 0.852
'fscore': 0.9, 'jac': 0.822, 'prec': 0.914, 'rec': 0.899

# Imports

In [1]:
import glob
import os
import time
import random
from PIL import Image
import logging
import shutil # to remove dirs


import pandas as pd

import torch
import torch.nn
from torch.autograd import Variable
import torchvision
from torch.utils.tensorboard import SummaryWriter

import segmentation_models_pytorch as smp

from torch.utils.data import DataLoader

import cv2

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, "helper")
from helper.dataset.meanteacher import *
# from helper.model.mean_teacher import * 
from helper.sampler.mixed_batch import *
# from helper.model.block.noise_block import GaussianNoiseBlock
from helper.compute.bin_seg import BCE_BinSeg_CU
from helper.compute.loss.shape import SignedDistanceLoss, HausdorffDTLoss
from helper.compute.loss.dice import DiceLoss
from helper.compute.metric.uncertainty import *
from helper.compute.metric.segmentation import *


#from pixelssl.utils import REGRESSION, CLASSIFICATION
#from pixelssl.utils import logger, cmd, tool
#from pixelssl.nn import func
#from pixelssl.nn.module import patch_replication_callback, GaussianNoiseLayer

C:\Users\Christina\anaconda3\envs\chrisy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Experiment Configs

In [16]:
class Configs():
    
    def __init__(self, reset=False):
        
        
        # =============================================================================
        # CHECK THESE
        # =============================================================================
        reset_2 = True # double control, otherwise all results for this prefix will be deleted
        if reset and reset_2:
            self.reset=True
        else:
            self.reset=False
        self.prefix = "exp8_clean_fda_b0_v2" # put local if christina's computer # exp7_baseline_try
        self.reduced_data = False
        self.dev_mode = False
        # "cuda" - use all the available GPUs
        # "cuda:1,3" - use specific GPUs            
        self.print_todos = False
        self.min_save_fscore = 0.7 # 0.7
        self.description = "b0 model, this time with all cirrus data, not sure i used all cirrus before"
        # =============================================================================
        # CHECK THESE
        # =============================================================================
        
        if "local" in self.prefix:
            self.device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"
            self.base_path = r"C:/Users/Prinzessin/projects/decentnet"
            self.logger_path = f"C:/Users/Prinzessin/projects/decentnet/datasceyence/results/{self.prefix}/logs"
            self.save_checkpoint_path = f"C:/Users/Prinzessin/projects/decentnet/datasceyence/results/{self.prefix}/ckpts"
        else:
            self.device = "cuda" # "cuda" if torch.cuda.is_available() else "cpu"
            self.base_path = r"C:/Users/Christina/Documents"
            self.logger_path = f"E:/Christina/datasceyence/results/{self.prefix}/logs"
            self.save_checkpoint_path = f"E:/Christina/datasceyence/results/{self.prefix}/ckpts"
        
        # smp unet ++ parameters
        self.encoder_name = "timm-efficientnet-b0" # "efficientnet-b7"
        self.encoder_weights = "imagenet"
        self.in_channels =  1
        self.n_output_neurons = 2
        
        self.eps = 1e-6
        
        self.num_workers = 16
        self.epochs = 150 # need more for final training ... more than 1000
        self.rampup_length = 300 # 600 for 1000
        
        # self.gaussian_noise = None # not in use
        
        self.ema_decay = 0.999 # default value
        
        # Sizes of tensors must match except in dimension 1
        # I solved the issue by resizing all the images size divisible to 32.
        self.image_size = 128 # 512
        
        # batch size = n_samples_per_class_per_batch * classes
        # for mixed batch sampling
        self.n_samples_per_class_per_batch = 4
        
        # self.lbs = 3 #  self.args.labeled_batch_size # .... remove this eventually and replace

        # optimisation
        # self.optimiser = "sgd"
        self.base_lr = 0.001 # 0.0001
        self.min_lr = 0.00001
        self.weight_decay = 1e-4 # not in use
        self.momentum = 0.9
        self.lr_update = 100 # should be roughly once per epoch
        
        # self.is_epoch_lrer = True # epoch or batch based learning rate updater
        
        # self.dropout = None
        
        # ablation studies
        self.clean_mean_teacher_ablation = True
        
        
         
        # =============================================================================
        # Paths
        # =============================================================================
        
        
        if not os.path.exists(self.base_path):
            os.makedirs(self.base_path)
        os.chdir(self.base_path) # this is now the main directory !!!!!!!!!!!!!!!!!!!!
        
        self.csv_filenames = glob.glob(r"datasceyence/data_prep/mt_*.csv")
        self.csv_test_filenames = ["datasceyence/data_prep/mt_data_cirrus.csv"] # mt_data_cirrus
        
        print(self.csv_filenames)
        
        # input # should be none for actual experiments
        self.load_checkpoint_file = None # r"E:\Christina\datasceyence\results\exp8_clean_fda_b7\ckpts\checkpoint_0.903_534.ckpt"
        # None # r"E:\Christina\datasceyence\results\exp7_clean_fda_try\ckpts\checkpoint_0.787_30.ckpt" # None # r"E:\Christina\datasceyence\results\exp6_with_tta_r200\ckpts\checkpoint_0.716_102.ckpt" # None # "E:/Christina/datasceyence/results/exp3_dice_cons_all_data_b7/ckpts/checkpoint_0.799_134.ckpt" 
        
        # all csv files used for run_mean_teacher.ipybn
        #self.csv_data_paths = [
        #    {"path" : r"data/data_ichallenge_amd.csv"}, 
        #    {"path" : r"data/data_ichallenge_non_amd.csv"}
        #]
        
        
        # output
        
        # if path exists: delete, create path
        if self.reset:
            if os.path.exists(self.logger_path):
                shutil.rmtree(self.logger_path)
            os.makedirs(self.logger_path)
            
        self.result_path = os.path.join(self.logger_path, f"metrics_{self.prefix}.csv" )
            
        
         # if path exists: delete, create path
        if self.reset:
            if os.path.exists(self.save_checkpoint_path):
                shutil.rmtree(self.save_checkpoint_path)
            os.makedirs(self.save_checkpoint_path)


    def log(self):
        # =============================================================================
        # save all class variables to file "configs.txt"
        # =============================================================================
        if self.reset:
            c = pd.DataFrame.from_dict({'key': self.__dict__.keys(), 'value': self.__dict__.values()})
            c.to_csv(os.path.join(self.logger_path, "configs2.txt"), sep=':', index=False)

# Routine

In [3]:
class RoutineMT:

    def __init__(self, configs):
        super(RoutineMT, self).__init__()
        
        self.configs = configs
                
        self.load_ckpt = torch.load(configs.load_checkpoint_file) if configs.load_checkpoint_file is not None else None
        
        self.i_item_total = 0
        
        
        # =============================================================================
        # Logger
        # =============================================================================
        
        self.writer =  SummaryWriter(log_dir=self.configs.logger_path)
        logging.basicConfig(filename=os.path.join(self.configs.logger_path, 'logger.log'), encoding='utf-8', level=logging.INFO) # everything more important or INFO
        # https://docs.python.org/3/library/logging.html
        
        # =============================================================================
        # Models
        # =============================================================================
        s_model = smp.UnetPlusPlus(
                        encoder_name=self.configs.encoder_name,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                        encoder_weights=self.configs.encoder_weights,  # use `imagenet` pre-trained weights for encoder initialization
                        in_channels=self.configs.in_channels,          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                        classes=self.configs.n_output_neurons,         # model output channels (number of classes in your dataset)
                    )
        
        t_model = smp.UnetPlusPlus(
                        encoder_name=self.configs.encoder_name,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                        encoder_weights=self.configs.encoder_weights,  # use `imagenet` pre-trained weights for encoder initialization
                        in_channels=self.configs.in_channels,          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                        classes=self.configs.n_output_neurons,         # model output channels (number of classes in your dataset)
                    )
        
        # detach the teacher model
        for param in t_model.parameters():
            param.detach_()
            
        # model= nn.DataParallel(model)
        # if only one gpu ??
        if "cpu" in self.configs.device:
            self.models = {'s': s_model, 
                           't': t_model}
        else: 
            self.models = {'s': torch.nn.DataParallel(s_model).to(self.configs.device), 
                           't': torch.nn.DataParallel(t_model).to(self.configs.device)}
        
        # add gaussian noise
        # currently not in use
        # self.gaussian_noiser = GaussianNoiseBlock(self.configs.gaussian_noise).cuda()
        
        # =============================================================================
        # Computing Units
        # =============================================================================
        self.computing_unit = {
            "s_train" : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="train", model="s", device=self.configs.device, writer=self.writer),
            "s_val"   : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="val", model="s", device=self.configs.device, writer=self.writer),
            "t_train" : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="train", model="t", device=self.configs.device, writer=self.writer),
            "s_test"  : BCE_BinSeg_CU(n_output_neurons=self.configs.n_output_neurons, mode="val", model="s", device=self.configs.device, writer=None),
        }
        
        self.top_val = { #'lowest_loss' : np.inf,  # low good
                         'highest_fscore' : self.configs.min_save_fscore, # high good
                         #'highest_jac' : 0, # high good
                         #'highest_symhd' : np.inf # low good
                     }
        
        
        
        # =============================================================================
        # Loss functions
        # =============================================================================
        # TODO: support more types of the consistency criterion
        # something with head and each head has a loss function attached??
        self.criterions = {'shape': HausdorffDTLoss(), # (device=self.configs.device, n_output_neurons=self.configs.n_output_neurons), # criterion_funcs[0](self.args),
                           'pixel': DiceLoss(n_output_neurons=self.configs.n_output_neurons, softmax=True), # added softmax on 29-may-2023 4pm  torch.nn.CrossEntropyLoss(), #
                           'cons_clean': DiceLoss(n_output_neurons=self.configs.n_output_neurons, softmax=True), # ,torch.nn.MSELoss() # softmax missing? there is a softmax mse loss ?? todo
                           'cons' : torch.nn.MSELoss(),
                          }
        
        # self.um = UncertaintyMetric(n_noise=4, n_repeat=2, n_output_neurons=self.configs.n_output_neurons)
        
        # =============================================================================
        # Datasets: train, val, test
        # =============================================================================
                
        train_set = MeanTeacherTrainDataset(mode="train", channels=self.configs.in_channels, image_size=self.configs.image_size, csv_filenames=self.configs.csv_filenames, reduced_data=self.configs.reduced_data)
        train_mbs = MixedBatchSampler(train_set.get_mbs_labels(), n_samples_per_class_per_batch=self.configs.n_samples_per_class_per_batch)
        
        print("*"*50)
        print("all labels", train_set.get_mbs_labels())
        
        x = train_set.get_mbs_labels()
        leastFrequent = min(x, key=x.count)
        amount = x.count(leastFrequent)
        print("this needs some error catching")
        self.steps = int(amount/self.configs.n_samples_per_class_per_batch)
        print("steps", self.steps)
                
        val_set = MeanTeacherValDataset(mode="val", channels=self.configs.in_channels, image_size=self.configs.image_size, csv_filenames=self.configs.csv_filenames, reduced_data=self.configs.reduced_data)
        
        cir_set = MeanTeacherCirDataset(channels=self.configs.in_channels, csv_filenames=self.configs.csv_test_filenames, image_size=self.configs.image_size)
              
        self.dataloader = {"train" : DataLoader(train_set, batch_sampler=train_mbs),
                           "val"   : DataLoader(val_set),
                           "cir"   : DataLoader(cir_set, shuffle=False)
                          }
        
        
        # =============================================================================
        # Optimisers
        # =============================================================================
        self.optims = {'s': torch.optim.SGD(self.models["s"].parameters(), lr=self.configs.base_lr, momentum=self.configs.momentum) # optimizer_funcs[0](self.models["s"].module.param_groups)
                          }

        # =============================================================================
        # Learning rate schedulers
        # =============================================================================
        self.lrsers = {'s': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optims["s"], 
                                                                              T_0 = self.configs.lr_update, # number of iterations for the first restart.
                                                                              eta_min = self.configs.min_lr
                                                                               )
                     }
                  
        # =============================================================================
        # Resume training
        # =============================================================================
        if self.load_ckpt:
            self.models["s"].load_state_dict(self.load_ckpt['s_model'])
            self.models["t"].load_state_dict(self.load_ckpt['t_model'])
            self.optims["s"].load_state_dict(self.load_ckpt['s_optim'])
            self.lrsers["s"].load_state_dict(self.load_ckpt['s_lrer'])
            try: 
                self.start_epoch = self.load_ckpt['epoch']
            except:
                self.start_epoch = self.load_ckpt['i_epoch']
        else:
            self.start_epoch = 0
            

    
    def run_training(self, i_epoch):
        # =============================================================================
        # Training
        # =============================================================================
        
        mode="train"
        
        self.models["s"].train()
        self.models["t"].eval() # train or eval ... ???
        
        for i_item, item in enumerate(self.dataloader[mode]):
            # =============================================================================
            # Process Batch
            # =============================================================================
            
            # get consistency weight
            def sigmoid_rampup(current, rampup_length):
                # calculate the ramp-up coefficient of the consistency constraint
                # returns vavlue between 0 and 1
                # https://github.com/HiLab-git/SSL4MIS/blob/master/code/utils/ramps.py
                # Consistency ramp-up from https://arxiv.org/abs/1610.02242
                if rampup_length == 0:
                    return 1.0
                else:
                    current = np.clip(current, 0.0, rampup_length)
                    phase = 1.0 - current / rampup_length
                    return float(np.exp(-5.0 * phase * phase))     
            cons_weight = sigmoid_rampup(i_epoch, self.configs.rampup_length)
            if self.writer:
                self.writer.add_scalars("info/weight", {"cons_ramp" : cons_weight}, self.i_item_total)
            
            
            
            # unlabelled for consistency loss
            unlabelled_batch_ids = np.where(item["has_mask"] == False) 
            # labelled for task loss
            labelled_batch_ids = np.where(item["has_mask"] == True) 
            
            # get predictions of student model for all images
            s_model_output = self.models["s"](item["img"].to(self.configs.device)) # we want all images (task loss and consistency loss)
            
            # run batch for student model
            self.computing_unit["s_train"].run_batch(criterions=self.criterions, model_output=s_model_output[labelled_batch_ids], ground_truth=item["msk"][labelled_batch_ids])
            
            if False:
                print(labelled_batch_ids)
                print(unlabelled_batch_ids)
                print("item"*40)
                print(item)
                print("item"*40)
                print("s model")
                print(s_model_output)
                print(item["msk"])
                print(s_model_output.shape)
                print(item["msk"].shape)
                print("next")

            # =============================================================================
            # Filter function
            # https://discuss.pytorch.org/t/how-does-applying-a-mask-to-the-output-affect-the-gradients/126520
            # =============================================================================
            def create_circle(model_output):
                    # 3 dims (batch, widht, height)
                    _, highest_class = torch.max(model_output, 1)  
                    # 3 dims (batch, widht, height)
                    final_masks = torch.zeros_like(highest_class)
                    #print(highest_class.shape)
                    for i, output in enumerate(highest_class):
                        output = output.detach().cpu().numpy()
                        #plt.figure()
                        #plt.imshow(output)
                        max_area = 0
                        circle = np.zeros(output.shape, dtype = np.uint8)
                        num_labels, labels_im = cv2.findContours(output, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)    
                        if len(num_labels) > 0:
                            max_area = max(num_labels, key = cv2.contourArea)
                            try:
                                ellipse = cv2.fitEllipse(max_area)
                                cv2.ellipse(circle, ellipse, (1), thickness=-1)
                                # cv2.drawContours(circle, [max_area], -1, color=(1), thickness=-1)
                                final_masks[i] = torch.tensor(circle)
                                #plt.figure()
                                #plt.imshow(circle)
                            except Exception as e:
                                logging.info("exception: could not fit a circle, use highest class instead")
                                # logging.info("fitting the ellipse didn't work")
                                logging.info("max area %d", cv2.contourArea(max_area))
                                logging.info(e)
                                #print("probably empty mask", cv2.contourArea(max_area))
                                #final_masks[i, 0] = model_output[i, 0]
                                final_masks[i] = highest_class[i]
                        else:
                            logging.info("could not find a connected component, use highest class instead")
                            #print(final_masks.shape)
                            #print(highest_class.shape)
                            #print(final_masks[i].shape)
                            #print(highest_class[i].shape)
                            # 2 dims + 1 = 3 dims
                            final_masks[i] = highest_class[i]
                    if self.configs.print_todos: 
                        print(final_masks.shape)
                    # 3 dims (batch, 1, widht, height)
                    final_masks = final_masks.unsqueeze(1)
                    if self.configs.print_todos: 
                        print(final_masks.shape)
                    return final_masks # Variable(final_masks, requires_grad=False) # circle
            # =============================================================================
            # Teacher Model
            # https://discuss.pytorch.org/t/how-does-applying-a-mask-to-the-output-affect-the-gradients/126520
            # =============================================================================
            
            #print(item["img"].shape)
            #print(item["img"][unlabelled_batch_ids].shape)
            
            no_grad_img_batch = Variable(item["img"], requires_grad=False)
            
            self.n_repeat = 2
            self.n_noise = 4
            
            # forward the teacher model
            with torch.no_grad():
                # get predictions of teacher model only for the labelled images
                t_model_output = self.models["t"](no_grad_img_batch.to(self.configs.device)) # we want all images (task loss and consistency loss)               
                # only labelled data
                self.computing_unit["t_train"].run_batch(
                                         criterions=self.criterions, 
                                         model_output=t_model_output[labelled_batch_ids], 
                                         ground_truth=item["msk"][labelled_batch_ids])
                
                
                if self.configs.clean_mean_teacher_ablation:
                    if len(no_grad_img_batch[unlabelled_batch_ids].shape) == 4:
                        batch_size, _, width, height = no_grad_img_batch[unlabelled_batch_ids].shape
                    elif len(no_grad_img_batch[unlabelled_batch_ids].shape) == 3:
                        batch_size, width, height = no_grad_img_batch[unlabelled_batch_ids].shape

                    cons_batch = no_grad_img_batch[self.get_centeroid].repeat(self.n_repeat, 1, 1 , 1)
                    batch_size_repeat = cons_batch.shape[0]
                    cons_preds = torch.zeros( (batch_size*self.n_noise*self.n_repeat), self.configs.n_output_neurons, width, height )

                    for i in range (self.n_noise):
                        noise = torch.clamp(torch.randn_like(cons_batch) * 0.1, min=-0.2, max=0.2)
                        cons_input = (cons_batch + noise)
                        cons_preds[ (self.n_repeat * batch_size * i) : (self.n_repeat * batch_size * (i+1)) ] = self.models["t"](cons_input.to(self.configs.device))

                    cons_preds = torch.nn.functional.softmax(cons_preds, dim=1)
                    cons_preds = cons_preds.reshape(self.n_repeat*self.n_noise, batch_size, self.configs.n_output_neurons, width, height)
                    
                    cons_preds_tmp = cons_preds.detach().cpu().numpy()
                    
                    cons_preds = torch.mean(cons_preds, dim=0)
                    #cons_preds = torch.sum(cons_preds, dim=0)
                    #cons_preds = torch.multiply(cons_preds, 0.75)

                    circle = create_circle(cons_preds)
                    t_pseudo_gt = circle.to(self.configs.device) # t_model_output * circle # Variable(t_model_output.detach().data, requires_grad=False) # t_model_output

                    entropy = -torch.sum(cons_preds * torch.log(cons_preds), dim=1, keepdim=True)
                    entropy = torch.abs(torch.mean(entropy)).to(self.configs.device)
                    
                
                    # _, t_pseudo_gt = torch.max(t_model_output, 1)
                    
                    
                                
                
                
            if False: 
                print("modeeeeeeeeeeeeeel output of teeeacheeer")
                #print(t_pseudo_gt)
                print(t_pseudo_gt.shape)
                print(" hier ist ein subplot")
                
                tmp_title = item['mbs_class'][labelled_batch_ids][0].detach().cpu().numpy()
                # print("lab", tmp_title)
                
                """
                fig, axs = plt.subplots(1, 4)
                _, highest_class = torch.max(s_model_output[labelled_batch_ids], 1)  
                highest_class = highest_class[0].detach().cpu().numpy()
                axs[0].imshow(highest_class, cmap="cool")
                axs[0].axis('off')
                axs[0].set_title("s model")
                
                _, highest_class2 = torch.max(t_model_output[labelled_batch_ids], 1)  
                highest_class2 = highest_class2[0].detach().cpu().numpy()
                axs[1].imshow(highest_class2, cmap="cool")
                axs[1].axis('off')
                axs[1].set_title("t model")
                
                axs[2].imshow(item["msk"][labelled_batch_ids][0,0].detach().cpu().numpy(), cmap="cool")
                axs[2].axis('off')
                
                axs[3].imshow(item["img"][labelled_batch_ids][0,0].detach().cpu().numpy(), cmap="gray")
                axs[3].axis('off')
                axs[3].set_title(str(tmp_title))
                
                tmp_title2 = item['mbs_class'][unlabelled_batch_ids][0].detach().cpu().numpy()
                #print("lab", tmp_title2)
                """
                
                fig, axs = plt.subplots(1, 6)
                _, highest_class = torch.max(s_model_output[unlabelled_batch_ids], 1)  
                highest_class = highest_class[0].detach().cpu().numpy()
                axs[0].imshow(highest_class, cmap="cool")
                axs[0].axis('off')
                axs[0].set_title("student")
                
                _, highest_class2 = torch.max(t_model_output[unlabelled_batch_ids], 1)  
                highest_class2 = highest_class2[0].detach().cpu().numpy()
                axs[1].imshow(highest_class2, cmap="cool")
                axs[1].axis('off')
                axs[1].set_title("teacher")
                
                axs[2].imshow(1-cons_preds_tmp[0,0,0], cmap="cool")
                axs[2].axis('off')
                axs[2].set_title("n1")
                
                axs[3].imshow(1-cons_preds[0,0].detach().cpu().numpy(), cmap="cool")
                axs[3].axis('off')
                axs[3].set_title("mean")
                
                axs[4].imshow(t_pseudo_gt[0,0].detach().cpu().numpy(), cmap="cool")
                axs[4].axis('off')
                axs[4].set_title("clean")
                
                #axs[4].imshow(item["msk"][unlabelled_batch_ids][0,0].detach().cpu().numpy(), cmap="cool")
                #axs[4].axis('off')
                #axs[4].set_title("mask")
                
                axs[5].imshow(item["img"][unlabelled_batch_ids][0,0].detach().cpu().numpy(), cmap="gray")
                axs[5].axis('off')
                axs[5].set_title('img')
                
                fig.savefig(f"{self.configs.prefix}_teacher{self.i_item_total}.jpg", dpi=1000)
                
            
        
                
            if unlabelled_batch_ids:
                # print(s_model_output[unlabelled_batch_ids].shape, t_pseudo_gt.shape)
                if self.configs.clean_mean_teacher_ablation:
                    cons_loss = self.criterions["cons_clean"](s_model_output[unlabelled_batch_ids], t_pseudo_gt) 
                else:
                    t_pseudo_gt = t_model_output[unlabelled_batch_ids]
                    cons_loss = self.criterions["cons"](s_model_output[unlabelled_batch_ids], t_pseudo_gt) 
                    entropy = torch.tensor(0).to(self.configs.device)
            else:
                cons_loss = self.zero_tensor
                            
            # this is a float
            #threshold = torch.tensor( ( 0.75 + (0.25*sigmoid_rampup(i_epoch, self.configs.rampup_length)) * np.log(2) ) ).to(self.configs.device)
            
            
            #if self.writer:
            #    self.writer.add_scalars("info/weight", {"thresh_ramp" : threshold.detach().cpu().numpy()}, self.i_item_total)
            
            #print(uncertainy_mask)
            #print(threshold)
            
            # this is not on the gpu ... now it is ...
            #bin_uncertainy_mask = (uncertainy_mask.to(self.configs.device) < threshold) # .float()
            
            #cons_loss_masked = torch.sum(bin_uncertainy_mask * cons_loss) / (2*torch.sum(bin_uncertainy_mask)+self.configs.eps)

            # =============================================================================
            # Backprop for student model
            # =============================================================================
            
            # combined loss
            # supervised loss + consistency loss * consistency weight
            if self.configs.print_todos: 
                print(" the losses ")
                print(cons_loss)
                print(entropy)
                print(self.computing_unit["s_train"].task_loss)
            
            loss = self.computing_unit["s_train"].task_loss + (cons_loss * cons_weight) + entropy #  + cons_loss_masked * cons_weight
            
            self.writer.add_scalars("info/loss", {"backprop" : loss,
                                                  "cons"  : cons_loss,
                                                  "cons_weighted"  : cons_loss * cons_weight,
                                                  "shape" : self.computing_unit["s_train"].shape_loss,
                                                  "pixel" : self.computing_unit["s_train"].pixel_loss}, self.i_item_total)
            
            self.optims["s"].zero_grad()
            loss.backward()
            self.optims["s"].step()      
            
            
            if self.writer:
                self.writer.add_scalars("info/uncertainty", {"entropy" : entropy.detach().cpu().numpy()}, self.i_item_total)
            
            # =============================================================================
            # LR Scheduler = every batch a step
            # =============================================================================
            self.lrsers["s"].step()
            if self.writer:
                self.writer.add_scalars("info/lr", {"s_train" : self.lrsers["s"].get_last_lr()[0]}, self.i_item_total)
            
            # =============================================================================
            # EMA for teacher model
            # https://github.com/HiLab-git/SSL4MIS/blob/master/code/train_uncertainty_aware_mean_teacher_2D.py
            # https://github.com/ZHKKKe/PixelSSL/blob/master/pixelssl/ssl_algorithm/ssl_mt.py
            # =============================================================================
            self.models["t"].train() # this should be train for ema
            
            self.i_item_total += 1
            
            local_ema_decay = min(1 - 1 / (self.i_item_total), self.configs.ema_decay)
            for t_param, s_param in zip(self.models["t"].parameters(), self.models["s"].parameters()):
                # model_weights = decay * model_weights + (1 - decay) * new_model_weights
                t_param.data = t_param.data * local_ema_decay +  s_param.data * (1 - local_ema_decay)
        
            
        
        # =============================================================================
        # Epoch Process (basically logging)
        # =============================================================================
        self.computing_unit["s_train"].run_epoch(i_epoch=i_epoch)
        self.computing_unit["t_train"].run_epoch(i_epoch=i_epoch)
        
        

        if self.configs.print_todos: 
            print("lr")
            print(self.lrsers["s"].get_last_lr())
            print(self.lrsers["s"].get_last_lr()[0])
        
        # =============================================================================
        # Epoch log and reset (training)
        # =============================================================================
        
        if self.configs.print_todos: 
            print("training task loss")
            print(self.computing_unit["s_train"].epoch_collector["taskloss"])

            print("training fscore")
            print(self.computing_unit["s_train"].epoch_collector["fscore"])
        
        self.computing_unit["s_train"].log(csv_file_path = self.configs.result_path)
        self.computing_unit["t_train"].log(csv_file_path = self.configs.result_path)
        self.computing_unit["s_train"].reset_epoch()
        self.computing_unit["t_train"].reset_epoch()
        
        
        
        
    def run_validation(self, i_epoch):
        self.models["s"].eval()
        
        # =============================================================================
        # for each batch
        # =============================================================================

        for i_item, item in enumerate(self.dataloader["val"]):
            # =============================================================================
            # Student
            # =============================================================================

            with torch.no_grad():
                # get predictions of student model for all images
                s_model_output = self.models["s"](item["img"].to(self.configs.device)) # we want all images (task loss and consistency loss)
                
                # should be based on loss function!!!!! - can we do all of them in the loss function? this is annoying
                # s_model_output = torch.nn.functional.softmax(s_model_output, dim=1) # I DON'T KNOOOOW

                # run batch for student model
                self.computing_unit["s_val"].run_batch(criterions=self.criterions, model_output=s_model_output, ground_truth=item["msk"])
        
        
        # =============================================================================
        # Epoch Process (basically logging)
        # =============================================================================
        self.computing_unit["s_val"].run_epoch(i_epoch=i_epoch)

        # =============================================================================
        # Epoch log and reset (validation)
        # =============================================================================
        
        if self.configs.print_todos: 
            print("validation task loss")
            print(self.computing_unit["s_val"].epoch_collector["taskloss"])

            print("validation fscore")
            print(self.computing_unit["s_val"].epoch_collector["fscore"])
        
        self.computing_unit["s_val"].log(csv_file_path = self.configs.result_path)
        self.computing_unit["s_val"].reset_epoch()
        
        logging.info("Validation of epoch %d done" % i_epoch)
        
    def run_final_evaluation(self, i_epoch):
                
        model = self.models["s"]
        
        paths = glob.glob(os.path.join(self.configs.save_checkpoint_path, f'checkpoint_*_{i_epoch}.ckpt'))
        
        model.load_state_dict(torch.load(paths[0])['s_model'])
        model.eval()

        for i_item, item in enumerate(self.dataloader["cir"]):
            
            

            with torch.no_grad():
                # get predictions of student model for all images
                s_model_output = self.models["s"](item["img"].to(self.configs.device)) # we want all images (task loss and consistency loss)
                
                img = item["img"].squeeze().detach().cpu().numpy()
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

                # run batch for student model
                self.computing_unit["s_test"].run_batch(criterions=self.criterions, model_output=s_model_output, ground_truth=item["msk"])
                
                # just works for batch size == 1 todo
                # for output in s_model_output:
                _, highest_class = torch.max(s_model_output, 1)  
                highest_class = highest_class[0].detach().cpu().numpy().astype(np.uint8)
                
                final_mask = np.zeros(highest_class.shape, dtype = np.uint8)
                num_labels, labels_im = cv2.findContours(highest_class, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)    
                if len(num_labels) > 0:
                    max_area = max(num_labels, key = cv2.contourArea)
                    try:
                        #ellipse = cv2.fitEllipse(max_area)
                        #cv2.ellipse(final_mask, ellipse, (1), thickness=-1)
                        cv2.drawContours(final_mask, [max_area], -1, color=(255), thickness=-1)
                        cv2.drawContours(img, [max_area], -1, color=(255, 255, 0), thickness=2)
                    except:
                        print("no ellipse possible")
                        pass
                
                
                from PIL import Image
                final_mask_pil = Image.fromarray(final_mask)
                p = item['img_path'][0].replace('enface_', 'enfaceDiscMask_').replace( "E:\Christina\Result_Data\CIRRUS_Normal\enface",    "E:\Christina\Result_Data\enfaceDiscMask_CIRRUS" ).replace( "E:\Christina\Result_Data\CIRRUS_Glaucoma\enface",    "E:\Christina\Result_Data\enfaceDiscMask_CIRRUS" )
                final_mask_pil.save(p) 
                
                
                final_mask_vis = img # Image.fromarray(img)
                p = item['img_path'][0].replace('enface_', 'enfaceDiscMaskVis_').replace( "E:\Christina\Result_Data\CIRRUS_Normal\enface",    "E:\Christina\Result_Data\enfaceDiscMask_CIRRUS" ).replace( "E:\Christina\Result_Data\CIRRUS_Glaucoma\enface",    "E:\Christina\Result_Data\enfaceDiscMask_CIRRUS" )   
                skimage.io.imsave(p, final_mask_vis)

                print("save mask: ", p)
            
                
                
                
                if False:
                    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
                    axs[0].imshow(item["img"][0].permute(1, 2, 0))
                    _, highest_class = torch.max(s_model_output, 1)  
                    print(highest_class.shape)
                    highest_class = highest_class[0].detach().cpu().numpy()
                    print(highest_class.shape)
                    axs[1].imshow(highest_class, cmap="cool")
                    axs[1].axis('off')
                    from skimage import measure
                    contours = measure.find_contours(highest_class, 0.5)

                    axs[2].imshow(item["img"][0].permute(1, 2, 0), cmap="gray")
                    for contour in contours:
                        axs[2].plot(contour[:, 1], contour[:, 0], linewidth=2, color = 'mediumvioletred')
                    contours2 = measure.find_contours(item["msk"][0].squeeze().detach().cpu().numpy(), 0.5)
                    for contour in contours2:    
                        axs[2].plot(contour[:, 1], contour[:, 0], linewidth=2, color = 'teal')
                    axs[2].axis('off')
                    axs[3].imshow(1-item["msk"][0].squeeze(), cmap="cool")
                    axs[3].axis('off')


                    fig.savefig(f"{self.configs.prefix}_result_{i_item}.jpg", dpi=1000)
                
                
            if self.dev_mode and i_item == 20:
                break
        
        
        # =============================================================================
        # Epoch Process (basically logging)
        # =============================================================================
        self.computing_unit["s_test"].run_epoch(i_epoch=i_epoch)

        # =============================================================================
        # Epoch log and reset (validation)
        # =============================================================================
        
        print("Test task loss")
        print(self.computing_unit["s_test"].epoch_collector["taskloss"])
        
        print("Test fscore")
        print(self.computing_unit["s_test"].epoch_collector["fscore"])
        
        
        
    def log(self, i_epoch):
        # =============================================================================
        # Save checkpoint
        # if epoch is greater than 2
        # if f-score improved - fscore initialised with min_save_fscore
        # =============================================================================
        
        if i_epoch > 2:

            if self.top_val['highest_fscore'] < self.computing_unit["s_val"].top_collector["highest_fscore"]:
                
                self.top_val['highest_fscore'] = self.computing_unit["s_val"].top_collector["highest_fscore"] 
                            
                checkpoint = {
                    'name': self.configs.prefix,
                    'i_epoch': i_epoch, 
                    's_model': self.models["s"].state_dict(),
                    't_model': self.models["t"].state_dict(),
                    's_optim': self.optims["s"].state_dict(),
                    's_lrer': self.lrsers["s"].state_dict()
                }
                
                tmp = self.top_val['highest_fscore']
                checkpoint_path = os.path.join(self.configs.save_checkpoint_path, f'checkpoint_{tmp}_{i_epoch}.ckpt')
                torch.save(checkpoint, checkpoint_path)

                logging.info("Saved model at epoch " + str(i_epoch) + " with fscore " + str(self.top_val['highest_fscore']))
            
        
        




# Run

In [4]:
if False:
    # Configs
    configs = Configs(reset=True)
    configs.log()

    # Run
    run = RoutineMT(configs)

    for i_epoch in range(run.start_epoch+1, configs.epochs):

        run.run_training(i_epoch=i_epoch)
        run.run_validation(i_epoch=i_epoch)
        run.log(i_epoch=i_epoch)
        print("*")

        if i_epoch > 1 and configs.reduced_data:
            break



['datasceyence/data_prep\\mt_data_cirrus.csv', 'datasceyence/data_prep\\mt_data_ichallenge_amd.csv', 'datasceyence/data_prep\\mt_data_plex.csv']
self.tgt_paths 0       E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
1       E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
2       E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
3       E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
4       E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
                              ...                        
1342    E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
1343    E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
1344    E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
1345    E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
1346    E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
Name: img_path, Length: 1347, dtype: object
**************************************************
all labels [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

FileNotFoundError: No such file: 'E:\Christina\Result_Data\ADAM_AMD\fundus\A0078.jpg'

In [17]:
# Configs
configs = Configs(reset=False)
configs.log()

# Run
run = RoutineMT(configs)

i_epoch = 116

run.run_final_evaluation(i_epoch=i_epoch)


['datasceyence/data_prep\\mt_data_cirrus.csv', 'datasceyence/data_prep\\mt_data_ichallenge_amd.csv', 'datasceyence/data_prep\\mt_data_plex.csv']


C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)


**************************************************
all labels [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\Christina\Documents\datasceyence\helper\compute\metric\segmentation.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  pixel_acc = (np.sum(matrix[0, :]) + self.eps) / (np.sum(matrix[0, :]) + np.sum(matrix[1, :]))


save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0362_Angiography 6x6 mm_7-16-2019_10-40-22_OD_sn11435_cube_z.tif
save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0515_Angiography 6x6 mm_3-24-2020_16-21-49_OS_sn7767_cube_z.tif
save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0074_Angiography 6x6 mm_9-4-2018_15-8-50_OS_sn0833_cube_z.tif
save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0198_Angiography 6x6 mm_1-15-2019_10-3-39_OS_sn2254_cube_z.tif
save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0306_Angiography 6x6 mm_5-27-2019_15-8-40_OS_sn9140_cube_z.tif
save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0285_Angiography 6x6 mm_4-29-2019_10-4-56_OD_sn8132_cube_z.tif
save mask:  E:\Christina\Result_Data\enfaceDiscMask_CIRRUS\enfaceDiscMaskVis_PSNASERI0187_Angiography 6x6 mm_1

In [15]:
# Configs
configs = Configs(reset=False)
configs.log()

# Run
run = RoutineMT(configs)

i_epoch = 116

run.run_final_evaluation(i_epoch=i_epoch)


['datasceyence/data_prep\\mt_data_cirrus.csv', 'datasceyence/data_prep\\mt_data_ichallenge_amd.csv', 'datasceyence/data_prep\\mt_data_plex.csv']


C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)
C:\Users\Christina\Documents\datasceyence\helper\dataset\template.py:39: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self.csv_data = pd.concat(csv_list, axis=0, ignore_index=False)


**************************************************
all labels [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [10]:
# tensorboard --logdir projects\decentnet\results\exp1_mt\logs port=6005   
# http://localhost:6005/?runColorGroup=regex%3A(_s_train|_s_val|_t_)#timeseries

# regex: (_s_train|_s_val|_t_)

# \exp6_with_tta_r200\ckpts

%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir E:/Christina/datasceyence/results/exp8_clean_fda_b0_v2/logs --port 6004

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorboard

experiment_id = "exp7_clean_fda_try"
experiment = tensorboard.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df
